In [1]:
import pandas as pd
import re
from datetime import datetime

### Countries

In [2]:
countries = pd.read_csv('exports/olympic_countries.csv', encoding='utf-8')
countries.head()

,country_name,country_code,country_3_letter_code
0,Andorra,AD,AND
1,United Arab Emirates,AE,UAE
2,Afghanistan,AF,AFG
3,Antigua and Barbuda,AG,ANT
4,Albania,AL,ALB


In [3]:
def sql_safe_value(value):
    """
    Transforme une valeur en une version SQL-safe :
    - Les chaînes de caractères sont entourées de guillemets simples avec échappement des guillemets simples internes.
    - Les valeurs manquantes (NaN) deviennent NULL.
    - Les valeurs numériques et NULL ne sont pas entourées de guillemets.
    - les subquery sont entourées de parenthèses
    """
    if pd.isna(value):
        return 'NULL'
    
    if isinstance(value, bool):
        return 'TRUE' if value else 'FALSE'
    
    # on détermine s'il s'agit d'une subquery en vérifiant si la valeur est de type SELECT .. FROM .. WHERE
    query_regex = re.compile(r'^SELECT .* FROM .* WHERE .*')
    if isinstance(value, str) and query_regex.match(value):
        return f"({value})"
    
    if isinstance(value, str):
        # Échapper les guillemets simples en doublant le guillemet simple
        safe_str = value.replace("'", "''")
        return f"'{safe_str}'"
    
    if isinstance(value, (int, float)):
        return str(value)
    
    raise ValueError(f"Type de donnée non pris en charge: {type(value)}")

In [4]:
def export_to_sql(idx: int, table_name: str, df: pd.DataFrame):
    """
    Exporte un DataFrame dans un fichier SQL.
    """
    with open(f'sql/{idx}_{table_name}.sql', 'w', encoding='utf-8') as f:
        # Réinitialiser l'index pour éviter les problèmes de désynchronisation
        df.reset_index(drop=True, inplace=True)

        # extraire les colonnes depuis le DataFrame
        columns = df.columns.tolist()
        f.write("START TRANSACTION;\n")
        f.write(f'INSERT INTO {table_name} ({", ".join(columns)}) VALUES\n')
        
        for i, row in df.iterrows():
            values = ', '.join(sql_safe_value(value) for value in row)
            f.write(f'({values})')
            
            if i < len(df) - 1:
                f.write(',')
            else:
                f.write(';')
            
            f.write('\n')

        f.write('COMMIT;')

In [5]:
def convert_to_mysql_datetime(date_str):
    '''Converti la date ISO 8601 en datetime'''
    dt = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
    mysql_datetime_str = dt.strftime('%Y-%m-%d %H:%M:%S')
    return mysql_datetime_str

In [6]:
# changer le nom des colonnes pour matcher avec la base de données
countries.rename(columns={
    'country_name': 'name',
    'country_code': 'code',
    'country_3_letter_code': 'code_iso'
}, inplace=True)

In [7]:
export_to_sql(1, 'countries', countries)

### Hosts

In [8]:
hosts = pd.read_csv('exports/olympic_hosts_cleaned.csv', encoding='utf-8')
hosts.head()

,index,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,People's Republic of China,Beijing 2022,Winter,2022
1,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


In [9]:
hosts.drop(columns=['index'], inplace=True)

In [10]:
hosts.rename(columns={
    'game_slug': 'slug',
    'game_start_date': 'start_date',
    'game_end_date': 'end_date',
    'game_location': 'location',
    'game_name': 'name',
    'game_season': 'season',
    'game_year': 'year'
}, inplace=True)

In [11]:
def location_subquery(location: str):
    return f"SELECT id FROM countries WHERE name = {sql_safe_value(location)}"

In [12]:
# traitements avant export
hosts['location'] = hosts['location'].apply(location_subquery)
hosts['start_date'] = hosts['start_date'].apply(convert_to_mysql_datetime)
hosts['end_date'] = hosts['end_date'].apply(convert_to_mysql_datetime)

In [13]:
export_to_sql(2, 'hosts', hosts)

### Disciplines

In [14]:
disciplines = pd.read_csv('exports/olympic_disciplines.csv', encoding='utf-8')
disciplines.head()

,discipline_title
0,Curling
1,Freestyle Skiing
2,Short Track Speed Skating
3,Snowboard
4,Ski Jumping


In [15]:
disciplines.rename(columns={
    'discipline_title': 'name'
}, inplace=True)

In [16]:
export_to_sql(3, 'disciplines', disciplines)

### Host_disciplines

In [17]:
results = pd.read_csv('exports/olympic_results_cleaned.csv', encoding='utf-8')
results.head()

,discipline_title,event_title,slug_game,participant_type,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,False,1,Italy,IT,ITA,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,False,1,Italy,IT,ITA,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,NaN,NaN
2,Curling,Mixed Doubles,beijing-2022,GameTeam,False,2,Norway,NO,NOR,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,NaN,NaN
3,Curling,Mixed Doubles,beijing-2022,GameTeam,False,2,Norway,NO,NOR,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,NaN,NaN
4,Curling,Mixed Doubles,beijing-2022,GameTeam,False,3,Sweden,SE,SWE,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,NaN,NaN


In [18]:
host_disciplines = results[['discipline_title', 'slug_game']].copy()

In [19]:
# supprimer les doublons
host_disciplines = host_disciplines.drop_duplicates()

In [20]:
# vérification des valeurs nulles
host_disciplines.isnull().sum()

discipline_title    0
slug_game           0
dtype: int64

In [21]:
def discipline_subquery(discipline: str):
    return f"SELECT id FROM disciplines WHERE name = {sql_safe_value(discipline)}"

host_disciplines['discipline_title'] = host_disciplines['discipline_title'].apply(discipline_subquery)

In [22]:
def game_subquery(slug: str):
    return f"SELECT id FROM hosts WHERE slug = {sql_safe_value(slug)}"

host_disciplines['slug_game'] = host_disciplines['slug_game'].apply(game_subquery)

In [23]:
host_disciplines.rename(columns={
    'discipline_title': 'id_discipline',
    'slug_game': 'id_host'
}, inplace=True)

In [24]:
export_to_sql(4, 'host_disciplines', host_disciplines)

### Events

In [25]:
events = pd.read_csv('exports/olympic_events.csv', encoding='utf-8')
events.head()

,event_title,discipline_title,slug_game,event_gender
0,Mixed Doubles,Curling,beijing-2022,Mixed
1,Women,Curling,beijing-2022,Women
2,Men,Curling,beijing-2022,Men
3,Men's Moguls,Freestyle Skiing,beijing-2022,Men
4,Men's Freeski Halfpipe,Freestyle Skiing,beijing-2022,Men


In [26]:
# Traitements avant export
events['discipline_title'] = events['discipline_title'].apply(discipline_subquery)
events['slug_game'] = events['slug_game'].apply(game_subquery)

In [27]:
events.rename(columns={
    'event_title': 'name',
    'discipline_title': 'id_discipline',
    'slug_game': 'id_host',
    'event_gender': 'gender'
}, inplace=True)

In [28]:
export_to_sql(5, 'events', events)

### Athletes

In [29]:
athletes = pd.read_csv('exports/olympic_athletes_cleaned.csv', encoding='utf-8')
athletes.head()

,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000,NaN,NaN
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995,NaN,NaN
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993,NaN,NaN
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995,NaN,NaN
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989,NaN,NaN


In [30]:
# on verifie si les valeurs de first_game ont une correspondance dans la table hosts
athletes_first_game = athletes['first_game'].unique()
hosts_name = hosts['name'].unique()

# différences
missing_hosts = set(athletes_first_game) - set(hosts_name)
missing_hosts

{'albertville-1992',
 'amsterdam-1928',
 'antwerp-1920',
 'athens-1896',
 'athens-2004',
 'atlanta-1996',
 'barcelona-1992',
 'beijing-2008',
 'beijing-2022',
 'berlin-1936',
 'calgary-1988',
 'chamonix-1924',
 'cortina-d-ampezzo-1956',
 'garmisch-partenkirchen-1936',
 'grenoble-1968',
 'helsinki-1952',
 'innsbruck-1964',
 'innsbruck-1976',
 'lake-placid-1932',
 'lake-placid-1980',
 'lillehammer-1994',
 'london-1908',
 'london-1948',
 'london-2012',
 'los-angeles-1932',
 'los-angeles-1984',
 'melbourne-1956',
 'mexico-city-1968',
 'montreal-1976',
 'moscow-1980',
 'munich-1972',
 'nagano-1998',
 nan,
 'oslo-1952',
 'paris-1900',
 'paris-1924',
 'pyeongchang-2018',
 'rio-2016',
 'rome-1960',
 'salt-lake-city-2002',
 'sapporo-1972',
 'sarajevo-1984',
 'seoul-1988',
 'sochi-2014',
 'squaw-valley-1960',
 'st-louis-1904',
 'st-moritz-1928',
 'st-moritz-1948',
 'stockholm-1912',
 'sydney-2000',
 'tokyo-1964',
 'tokyo-2020',
 'turin-2006',
 'vancouver-2010'}

In [31]:
slug_to_name = hosts.set_index('slug')['name'].to_dict()

def resolve_first_game(first_game: str):
    if first_game in missing_hosts and first_game in slug_to_name:
        return slug_to_name[first_game]
    return first_game

athletes['first_game'] = athletes['first_game'].apply(resolve_first_game)

In [32]:
# on vérifie si les modifcations ont été bien effectuées
athletes_first_game = athletes['first_game'].unique()
missing_hosts = set(athletes_first_game) - set(hosts_name)
missing_hosts

{nan}

In [33]:
# vérification des valeurs nulles
athletes.isnull().sum()

athlete_url              7946
athlete_full_name           1
games_participations        0
first_game                 23
athlete_year_birth          0
athlete_medals          68523
bio                     61033
dtype: int64

In [34]:
# suppression des colonnes inutiles
athletes.drop(columns=['games_participations', 'athlete_medals'], inplace=True)

In [35]:
def game_subquery_by_name(name: str):
    return f"SELECT id FROM hosts WHERE name = {sql_safe_value(name)}"

In [36]:
# Traitements avant export
athletes['first_game'] = athletes['first_game'].apply(game_subquery_by_name)
athletes['athlete_year_birth'] = athletes['athlete_year_birth'].replace(0, pd.NA)

In [37]:
athletes.rename(columns={
    'athlete_full_name': 'full_name',
    'athlete_year_birth': 'birth_year',
    'first_game': 'first_edition',
    'athlete_url': 'url',
}, inplace=True)

In [38]:
athletes.loc[athletes['full_name'].isna()]

,url,full_name,first_edition,birth_year,bio
75904,NaN,NaN,SELECT id FROM mia15.hosts WHERE name = NULL,<NA>,NaN


In [39]:
# supprimer si full_name est nan
athletes = athletes.dropna(subset=['full_name'])

In [40]:
export_to_sql(6, 'athletes', athletes)

### Results

In [41]:
results = pd.read_csv('exports/olympic_results_cleaned.csv', encoding='utf-8')
results.head()

,discipline_title,event_title,slug_game,participant_type,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,False,1,Italy,IT,ITA,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,False,1,Italy,IT,ITA,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,NaN,NaN
2,Curling,Mixed Doubles,beijing-2022,GameTeam,False,2,Norway,NO,NOR,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,NaN,NaN
3,Curling,Mixed Doubles,beijing-2022,GameTeam,False,2,Norway,NO,NOR,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,NaN,NaN
4,Curling,Mixed Doubles,beijing-2022,GameTeam,False,3,Sweden,SE,SWE,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,NaN,NaN


In [42]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170636 entries, 0 to 170635
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   discipline_title       170636 non-null  object
 1   event_title            170636 non-null  object
 2   slug_game              170636 non-null  object
 3   participant_type       170636 non-null  object
 4   rank_equal             33336 non-null   object
 5   rank_position          166757 non-null  object
 6   country_name           170636 non-null  object
 7   country_code           165398 non-null  object
 8   country_3_letter_code  170636 non-null  object
 9   athlete_url            145050 non-null  object
 10  athlete_full_name      157481 non-null  object
 11  value_unit             82795 non-null   object
 12  value_type             82795 non-null   object
dtypes: object(13)
memory usage: 16.9+ MB


In [43]:
# vérifier les valeurs nulles
results['rank_equal'].isna().sum()

137300

In [44]:
# remplacer par False les valeurs nulles
results['rank_equal'] = results['rank_equal'].fillna(False)

/tmp/ipykernel_64878/148233537.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results['rank_equal'] = results['rank_equal'].fillna(False)


In [45]:
# mettre nan les rank position qui ne sont pas des entiers
results['rank_position'] = pd.to_numeric(results['rank_position'], errors='coerce').astype('Int64')

In [46]:
def athlete_subquery(x: pd.Series):
    # NOTE: ça pose problème à cause des homonymes, il faut utiliser l'url quand c'est possible
    if not pd.isna(x['athlete_url']):
        return f"SELECT id FROM athletes WHERE url = {sql_safe_value(x['athlete_url'])}"
    
    return f"SELECT id FROM athletes WHERE full_name = {sql_safe_value(x['athlete_full_name'])}"

# id_athlete
results['athlete_full_name'] = results[['athlete_full_name', 'athlete_url']].apply(athlete_subquery, axis=1)

In [47]:
def event_subquery(e: pd.Series):
    return f"""SELECT id FROM events WHERE name = {sql_safe_value(e['event_title'])} 
    and id_discipline = ({discipline_subquery(e['discipline_title'])}) 
    and id_host = ({game_subquery(e['slug_game'])})"""

# id_event
results['event_title'] = results[['discipline_title', 'event_title', 'slug_game']].apply(event_subquery, axis=1)

In [48]:
# id_country
results['country_name'] = results['country_name'].apply(location_subquery)

In [49]:
results.rename(columns={
    'event_title': 'id_event',
    'participant_type': 'participant_type',
    'value_unit': 'value',
    'value_type': 'value_type',
    'rank_position': 'position',
    'rank_equal': 'is_equality',
    'country_name': 'id_country',
    'athlete_full_name': 'id_athlete'
}, inplace=True)

# drop toutes les autres colonnes
results.drop(columns=['discipline_title', 'slug_game', 'athlete_url', 'country_code', 'country_3_letter_code'], inplace=True)

In [50]:
export_to_sql(7, 'results', results)